# Reading data online

This notebooks contains examples on how to read climate data from IRI library and from NOAA PSL. This recipe applies to any other data provider that supports the OPeNDAP protocol.

First we need to import the required libraries

In [1]:
import xarray as xr

As a rule of thumb, most of the data that is downloaded or available from IRI library will have an issue with the time dimension. This is because the calendar used by IRI library needs to be properly encoded in the data attributes.

## SODA - IRI library

SODA data comes with some calendar issues that need to be fixed.

In [2]:
soda_data = xr.open_dataset(
    "http://iridl.ldeo.columbia.edu/SOURCES/.CARTON/.SODA3/.SODA3.3.1/.temp/dods",
    decode_cf=False,
)
soda_data["time"].attrs["calendar"] = "360_day"
soda_data = xr.decode_cf(soda_data)
soda_data

<xarray.Dataset> Size: 21GB
Dimensions:    (longitude: 720, depth: 50, latitude: 330, time: 432)
Coordinates:
  * longitude  (longitude) float32 3kB 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
  * depth      (depth) float32 200B 5.034 15.1 25.22 ... 5.185e+03 5.395e+03
  * latitude   (latitude) float32 1kB -74.75 -74.25 -73.75 ... 88.75 89.25 89.75
  * time       (time) object 3kB 1980-01-16 00:00:00 ... 2015-12-16 00:00:00
Data variables:
    temp       (time, depth, latitude, longitude) float32 21GB ...
Attributes:
    Conventions:  IRIDL

Time and spatial selection can be performed either in the code or in the URL.

In [3]:
# Code
soda_data.sel(
    time=slice("1980-01", "1980-12"), latitude=slice(-10, 10), longitude=slice(120, 280)
)

<xarray.Dataset> Size: 31MB
Dimensions:    (longitude: 320, depth: 50, latitude: 40, time: 12)
Coordinates:
  * longitude  (longitude) float32 1kB 120.2 120.8 121.2 ... 278.8 279.2 279.8
  * depth      (depth) float32 200B 5.034 15.1 25.22 ... 5.185e+03 5.395e+03
  * latitude   (latitude) float32 160B -9.75 -9.25 -8.75 ... 8.75 9.25 9.75
  * time       (time) object 96B 1980-01-16 00:00:00 ... 1980-12-16 00:00:00
Data variables:
    temp       (time, depth, latitude, longitude) float32 31MB ...
Attributes:
    Conventions:  IRIDL

The URL selection method can be a bit tricky to work with, so I recommend using the code selection method. You can use the IRI Library data selection tool to get the final URL and then use it in the code. This also expands the possibilities of data via URL manipulation.

In [4]:
xr.open_dataset(
    "http://iridl.ldeo.columbia.edu/SOURCES/.CARTON/.SODA3/.SODA3.3.1/.temp/time/%28Jan%201980%29%28Dec%201980%29RANGEEDGES/dods",
    decode_cf=False,
)

<xarray.Dataset> Size: 570MB
Dimensions:    (longitude: 720, depth: 50, latitude: 330, time: 12)
Coordinates:
  * longitude  (longitude) float32 3kB 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
  * depth      (depth) float32 200B 5.034 15.1 25.22 ... 5.185e+03 5.395e+03
  * latitude   (latitude) float32 1kB -74.75 -74.25 -73.75 ... 88.75 89.25 89.75
  * time       (time) float32 48B 240.5 241.5 242.5 243.5 ... 249.5 250.5 251.5
Data variables:
    temp       (time, depth, latitude, longitude) float32 570MB ...
Attributes:
    Conventions:  IRIDL

# ERSSTv5

Most other datasets on different servers will have a similar structure. The following example shows how to read the ERSSTv5 data from NOAA PSL. You have to search for the OPeNDAP/DAP/THREDDS/dods link on the data page and use that link to read the data.

In [5]:
ersstv5_data = xr.open_dataset(
    "http://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
)
ersstv5_data

<xarray.Dataset> Size: 132MB
Dimensions:    (lat: 89, lon: 180, time: 2054, nbnds: 2)
Coordinates:
  * lat        (lat) float32 356B 88.0 86.0 84.0 82.0 ... -84.0 -86.0 -88.0
  * lon        (lon) float32 720B 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 16kB 1854-01-01 1854-02-01 ... 2025-02-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 33kB ...
    sst        (time, lat, lon) float32 132MB ...
Attributes: (12/39)
    climatology:                     Climatology is based on 1971-2000 SST, X...
    description:                     In situ data: ICOADS2.5 before 2007 and ...
    keywords_vocabulary:             NASA Global Change Master Directory (GCM...
    keywords:                        Earth Science > Oceans > Ocean Temperatu...
    instrument:                      Conventional thermometers
    source_comment:                  SSTs were observed by conventional therm...
    ...                              ...
    comment:                         SSTs were observed by conventional therm...
    summary:                         ERSST.v5 is developed based on v4 after ...
    dataset_title:                   NOAA Extended Reconstructed SST V5
    _NCProperties:                   version=2,netcdf=4.6.3,hdf5=1.10.5
    data_modified:                   2025-03-04
    DODS_EXTRA.Unlimited_Dimension:  time